## Import Libraries

In [1]:
from vllm import LLM, SamplingParams
import tqdm as notebook_tqdm
import json
import os
import re
import copy
from huggingface_hub import login

with open('/home/nd2794/keys.json', 'r') as file:
    keys = json.load(file)

cache_dir = '/proj/vondrick4/naveen/HF_CACHE_DIR'
hf_token = keys["HF_READ_TOKEN"]
login(token=hf_token)

os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HUGGINGFACE_HUB_CACHE'] = cache_dir
os.environ['PYTORCH_TRANSFORMERS_CACHE'] = cache_dir
os.environ['PYTORCH_PRETRAINED_BERT_CACHE'] = cache_dir
os.environ['HF_HUB_CACHE'] = cache_dir

/proj/vondrick2/naveen/miniconda3/envs/tr-vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-06 10:15:37,383	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Define Utility Functions

In [2]:
def get_instruction(source_event, target_event):
    common_instruct = """Two images (Source & Target) are described by their captions.
Your have generate a query text for composed image retrieval task that describes how the Source image should be modified to get the Target image.
Each image may multiple captions which are listed in a new line. You must read all the captions for each image to fully understand the image.
Only output the query text."""

    source_instruct = """SOURCE IMAGE CAPTIONS:\n""" + "\n".join(f"- {item}" for item in source_event)

    target_instruct = """TARGET IMAGE CAPTIONS:\n""" + "\n".join(f"- {item}" for item in target_event)

    return common_instruct + '\n\n' + source_instruct + '\n\n' + target_instruct

In [3]:
ex_source_cap = [
    'A person in blue and orange shirt riding a bicycle.',
    'A bicyclist riding in front of speed blurred foliage. ',
    'A cyclist with a helmet bicycles quickly on a road.',
    'The man is riding his bike down the street. '
]

ex_target_cap = [
    'A man is riding a motorcycle down the street',
    'A man riding a motorcycle down a street next to a restaurant.',
    'Helmeted motorcyclist riding on roadway in populated setting. ',
    'A motorcycle going down a street very fast.'
]

ex_query = "The person is riding a motorcycle"

In [4]:
conversation_template = [
    {
        "role": "system",
        "content": "You are a helpful assistant"
    },
    {
        "role": "user",
        "content": get_instruction(ex_source_cap, ex_target_cap)
    },
    {
        "role": "assistant",
        "content": ex_query 
    }
]

## Load Model

In [5]:
model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
llm = LLM(
    model=model_name, 
    tensor_parallel_size=4, 
    download_dir=cache_dir,
    dtype='bfloat16'
)

INFO 11-06 10:15:48 config.py:905] Defaulting to use mp for distributed inference
WARNING 11-06 10:15:48 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-06 10:15:48 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-06 10:15:48 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir='/proj/vondrick4/naveen/HF_CACHE_DIR', load_format=LoadFormat.AUTO, tensor_parall

[W1106 10:15:51.984727320 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [::ffff:127.0.0.1]:35777 (errno: 97 - Address family not supported by protocol).
[W1106 10:15:51.432841821 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [::ffff:127.0.0.1]:35777 (errno: 97 - Address family not supported by protocol).
[W1106 10:15:51.464829314 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [::ffff:127.0.0.1]:35777 (errno: 97 - Address family not supported by protocol).
[W1106 10:15:51.632998437 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [::ffff:127.0.0.1]:35777 (errno: 97 - Address family not supported by protocol).


INFO 11-06 10:15:51 utils.py:1008] Found nccl from library libnccl.so.2
INFO 11-06 10:15:51 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=1392604) (VllmWorkerProcess pid=1392605) (VllmWorkerProcess pid=1392606) INFO 11-06 10:15:51 utils.py:1008] Found nccl from library libnccl.so.2
INFO 11-06 10:15:51 utils.py:1008] Found nccl from library libnccl.so.2
INFO 11-06 10:15:51 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=1392604) (VllmWorkerProcess pid=1392606) (VllmWorkerProcess pid=1392605) INFO 11-06 10:15:51 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 11-06 10:15:51 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 11-06 10:15:51 pynccl.py:63] vLLM is using nccl==2.20.5
WARNING 11-06 10:15:52 custom_all_reduce.py:132] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
(VllmWorkerProcess pid=1392605) (VllmWorkerProcess pid=1392604

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  3.39it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  3.32it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:00<00:00,  4.55it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.61it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.67it/s]



INFO 11-06 10:15:54 model_runner.py:1067] Loading model weights took 3.7710 GB
(VllmWorkerProcess pid=1392604) INFO 11-06 10:15:54 model_runner.py:1067] Loading model weights took 3.7710 GB
(VllmWorkerProcess pid=1392606) INFO 11-06 10:15:54 model_runner.py:1067] Loading model weights took 3.7710 GB
(VllmWorkerProcess pid=1392605) INFO 11-06 10:15:54 model_runner.py:1067] Loading model weights took 3.7710 GB
INFO 11-06 10:15:55 distributed_gpu_executor.py:57] # GPU blocks: 76065, # CPU blocks: 8192
INFO 11-06 10:15:55 distributed_gpu_executor.py:61] Maximum concurrency for 131072 tokens per request: 9.29x
INFO 11-06 10:15:57 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-06 10:15:57 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decr

In [7]:
sampling_params = SamplingParams(
    #temperature=1.0,
    max_tokens=200,
)

## Load the data

In [44]:
split = 'train'
if split == 'train':
    data = json.load(open('/proj/vondrick4/naveen/coir-data/LaSCo/metadata/lasco_train_indexed_MT(img_cap).json', 'r'))
    imgs_and_caps = json.load(open('/proj/vondrick4/naveen/coir-data/annotations/images_and_caps_train2014_processed_dict.json', 'r'))
elif split == 'val':
    data = json.load(open('/proj/vondrick4/naveen/coir-data/LaSCo/metadata/lasco_val_indexed_MT(img_cap).json', 'r'))
    imgs_and_caps= json.load(open('/proj/vondrick4/naveen/coir-data/annotations/images_and_caps_val2014_processed_dict.json', 'r'))
    for key in imgs_and_caps.keys():
        imgs_and_caps[key] = [imgs_and_caps[key]]

## Process the data

In [45]:
conversations = []
tracker = {}
i = 0

for sample in data:
    tracker[sample['id']] = {}
    
    conversation_forward = copy.deepcopy(conversation_template)
    conversation_forward.append(
        {
            "role": "user",
            "content": get_instruction(imgs_and_caps[str(sample["coir"]["query-image-id"])], imgs_and_caps[str(sample["coir"]["target-image-id"])])
        }
    )
    
    conversation_reverse = copy.deepcopy(conversation_template)
    conversation_reverse.append(
        {
            "role": "user",
            "content": get_instruction(imgs_and_caps[str(sample["coir"]["target-image-id"])], imgs_and_caps[str(sample["coir"]["query-image-id"])])
        }
    )
    
    conversations.append(conversation_forward)
    tracker[sample['id']]['forward'] = i
    i+=1
    
    conversations.append(conversation_reverse)
    tracker[sample['id']]['reverse'] = i
    i+=1

## Run vLLM

In [46]:
outputs = llm.chat(
    messages=conversations,
    sampling_params=sampling_params,
    use_tqdm=True
)

Processed prompts: 100%|██████████████████████████████████████████████████████████████████████████| 200/200 [00:07<00:00, 26.59it/s, est. speed input: 9340.37 toks/s, output: 445.31 toks/s]


## Process & Write Outputs

In [47]:
processed_outputs = []
for out in outputs:
    processed_outputs.append(out.outputs[0].text)

In [48]:
for sample in data:
    sample["coir"]['query-text-forward-mg'] = processed_outputs[tracker[sample['id']]['forward']]
    sample["coir"]['query-text-reverse-mg'] = processed_outputs[tracker[sample['id']]['reverse']]

In [49]:
with open('/proj/vondrick4/naveen/coir-data/LaSCo/metadata/lasco_{}_indexed_MT(img_cap)_MG.json'.format(split), 'w') as json_file:
    json.dump(data, json_file, indent=4)

ERROR 11-06 10:39:17 multiproc_worker_utils.py:116] Worker VllmWorkerProcess pid 1392604 died, exit code: -15
INFO 11-06 10:39:17 multiproc_worker_utils.py:120] Killing local vLLM worker processes
